In [1]:
import os
import glob
import numpy as np
from PIL import Image 
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms

In [2]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),           
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

In [5]:
train_dir = '/kaggle/input/baldnotbald/Train/Train'
test_dir = '/kaggle/input/baldnotbald/Test/Test'

In [6]:
train_dataset = dataset.ImageFolder(root=train_dir, transform=transform)
test_dataset = dataset.ImageFolder(root=test_dir, transform=transform)

In [7]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 5672
    Root location: /kaggle/input/baldnotbald/Train/Train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [8]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2087
    Root location: /kaggle/input/baldnotbald/Test/Test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [9]:
print(train_dataset.classes)

['Bald', 'NotBald']


In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [11]:
len(train_dataloader)

89

In [12]:
len(test_dataloader)

33

In [13]:
model = torchvision.models.resnet18(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 123MB/s] 


In [14]:
in_features = model.fc.in_features
out_features = 2

class SoftmaxClassifier(nn.Module):
    def __init__(self, in_features, out_features):
        super(SoftmaxClassifier, self).__init__()
        self.fc = nn.Linear(in_features, out_features)

    def forward(self, x):
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x

model.fc = SoftmaxClassifier(in_features=in_features, out_features=out_features)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
model.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

In [17]:
for epoch in range(20):
    model.train()
    print("Training")
    running_loss = 0
    correct = 0
    total = 0
    
    for data in train_dataloader:
        img, labels = data
        
        img = img.to('cuda')
        labels = labels.to('cuda')
        
        optimizer.zero_grad()
        
        outputs = model(img)
        #pred = pred.to('cpu')
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        #total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        running_loss += loss.item()
        
    train_loss = running_loss/(len(train_dataloader)*64)
    train_acc = correct/(len(train_dataloader)*64)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc*100:.2f}%")
    

Training
Epoch 1, Train Loss: 0.0080, Train Accuracy: 82.57%
Training
Epoch 2, Train Loss: 0.0068, Train Accuracy: 88.29%
Training
Epoch 3, Train Loss: 0.0067, Train Accuracy: 89.36%
Training
Epoch 4, Train Loss: 0.0065, Train Accuracy: 89.94%
Training
Epoch 5, Train Loss: 0.0065, Train Accuracy: 90.41%
Training
Epoch 6, Train Loss: 0.0064, Train Accuracy: 90.54%
Training
Epoch 7, Train Loss: 0.0064, Train Accuracy: 90.41%
Training
Epoch 8, Train Loss: 0.0064, Train Accuracy: 90.54%
Training
Epoch 9, Train Loss: 0.0063, Train Accuracy: 90.64%
Training
Epoch 10, Train Loss: 0.0063, Train Accuracy: 90.55%
Training
Epoch 11, Train Loss: 0.0063, Train Accuracy: 90.78%
Training
Epoch 12, Train Loss: 0.0063, Train Accuracy: 91.15%
Training
Epoch 13, Train Loss: 0.0063, Train Accuracy: 90.85%
Training
Epoch 14, Train Loss: 0.0062, Train Accuracy: 91.15%
Training
Epoch 15, Train Loss: 0.0062, Train Accuracy: 91.70%
Training
Epoch 16, Train Loss: 0.0062, Train Accuracy: 91.45%
Training
Epoch 17

In [18]:
checkpoint_path = '/kaggle/working/model_bald_checkpoint.pth' 

In [20]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': train_loss,
    'accuracy': train_acc,
}, checkpoint_path)
